In [1]:
path = '/Users/FerdinandYu/spark'
import findspark
findspark.init(path)

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("dbm_project") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

In [2]:
yelp = spark.read.json("/Users/FerdinandYu/Downloads/mini_review.json")

In [3]:
yelp.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [4]:
yelp = yelp.select(yelp['review_id'], yelp['date'], yelp['business_id'], yelp['user_id'], yelp['stars'])

In [5]:
yelp.createOrReplaceTempView("yelp_view")

In [6]:
spark.sql("select user_id, row_number() over(order by user_id asc) user_index from (select distinct user_id from yelp_view) a").createOrReplaceTempView("users")

In [7]:
spark.sql("select business_id, row_number() over(order by business_id asc) business_index from (select distinct business_id from yelp_view) a").createOrReplaceTempView("businesses")

In [8]:
spark.sql("select user_id, business_id, stars from (SELECT user_id, business_id, stars, row_number() over(partition by user_id, business_id order by date asc) rk FROM yelp_view) a where rk = 1").createOrReplaceTempView("yelp_first")

In [10]:
df = spark.sql("select b.user_index, c.business_index, a.stars\
               from yelp_first a, users b, businesses c\
               where a.user_id = b.user_id and a.business_id = c.business_id")

In [11]:
type(df)

pyspark.sql.dataframe.DataFrame

In [12]:
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry

In [13]:
cmat=CoordinateMatrix(df.rdd.map(tuple))

In [15]:
cmat.entries.collect()

[MatrixEntry(5279, 241, 1.0),
 MatrixEntry(40690, 241, 5.0),
 MatrixEntry(32306, 241, 5.0),
 MatrixEntry(47171, 241, 1.0),
 MatrixEntry(11540, 452, 2.0),
 MatrixEntry(64961, 530, 5.0),
 MatrixEntry(6680, 530, 4.0),
 MatrixEntry(27657, 530, 4.0),
 MatrixEntry(22541, 530, 5.0),
 MatrixEntry(36043, 530, 4.0),
 MatrixEntry(43962, 530, 4.0),
 MatrixEntry(59325, 530, 4.0),
 MatrixEntry(18088, 530, 4.0),
 MatrixEntry(6792, 530, 4.0),
 MatrixEntry(50633, 530, 5.0),
 MatrixEntry(35296, 530, 2.0),
 MatrixEntry(54953, 530, 3.0),
 MatrixEntry(75468, 530, 4.0),
 MatrixEntry(78246, 1193, 5.0),
 MatrixEntry(53362, 1206, 1.0),
 MatrixEntry(22769, 1206, 1.0),
 MatrixEntry(76795, 1288, 5.0),
 MatrixEntry(52029, 1578, 3.0),
 MatrixEntry(16744, 1578, 5.0),
 MatrixEntry(25256, 1578, 4.0),
 MatrixEntry(59178, 1578, 4.0),
 MatrixEntry(3469, 1637, 4.0),
 MatrixEntry(15793, 1637, 3.0),
 MatrixEntry(51146, 1797, 5.0),
 MatrixEntry(11003, 1844, 4.0),
 MatrixEntry(41720, 1924, 5.0),
 MatrixEntry(44720, 1924, 5.0)

In [17]:
df.show()

+----------+--------------+-----+
|user_index|business_index|stars|
+----------+--------------+-----+
|      5279|           241|  1.0|
|     40690|           241|  5.0|
|     32306|           241|  5.0|
|     47171|           241|  1.0|
|     11540|           452|  2.0|
|     64961|           530|  5.0|
|      6680|           530|  4.0|
|     27657|           530|  4.0|
|     22541|           530|  5.0|
|     36043|           530|  4.0|
|     43962|           530|  4.0|
|     59325|           530|  4.0|
|     18088|           530|  4.0|
|      6792|           530|  4.0|
|     50633|           530|  5.0|
|     35296|           530|  2.0|
|     54953|           530|  3.0|
|     75468|           530|  4.0|
|     78246|          1193|  5.0|
|     53362|          1206|  1.0|
+----------+--------------+-----+
only showing top 20 rows



In [18]:
from scipy import sparse
import numpy as np

rows = np.concatenate(
        df.select("user_index").rdd.glom().map(
          lambda x: np.array([elem[0] for elem in x]))
        .collect())

cols = np.concatenate(
        df.select("business_index").rdd.glom().map(
          lambda x: np.array([elem[0] for elem in x]))
        .collect())

data = np.concatenate(
        df.select("stars").rdd.glom().map(
          lambda x: np.array([elem[0] for elem in x]))
        .collect())

sparse_matrix = sparse.coo_matrix((data, (rows, cols)), 
                    shape=(num_users, num_items))

sparse_matrix = sparse_matrix.tocsr()

NameError: name 'num_users' is not defined